In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-intern-data/cleaned_intern_data.csv


In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
api_key= user_secrets.get_secret("PINECONE_API_KEY")


**creating a single, unified embedding for each product that represents both its textual description and its visual appearance.**

In [3]:
import pandas as pd
import numpy as np

# Load the cleaned dataset
df = pd.read_csv('/kaggle/input/cleaned-intern-data/cleaned_intern_data.csv')
print(df.head())

                                               title            brand  \
0  GOYMFK 1pc Free Standing Shoe Rack, Multi-laye...           GOYMFK   
1  Plant Repotting Mat MUYETOL Waterproof Transpl...          MUYETOL   
2  Pickleball Doormat, Welcome Doormat Absorbent ...          VEWETOL   
3  JOIN IRON Foldable TV Trays for Eating Set of ...  JOIN IRON Store   
4  Folews Bathroom Organizer Over The Toilet Stor...     Folews Store   

                                         description  price  \
0  multiple shoes, coats, hats, and other items E...  24.99   
1  Plant Repotting Mat MUYETOL Waterproof Transpl...   5.98   
2  The decorative doormat features a subtle textu...  13.99   
3  Set of Four Folding Trays With Matching Storag...  89.99   
4  Folews Bathroom Organizer Over The Toilet Stor...  63.99   

                                          categories  \
0  ['Home & Kitchen', 'Storage & Organization', '...   
1  ['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...   
2  ['Patio, La

In [4]:
#!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
# Load the pre-trained model
text_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# Create a combined text field for embedding. This is our key feature engineering step.
df['combined_text'] = (
    df['title'].fillna('') + '. ' +
    df['description'].fillna('') + '. ' +
    df['categories'].fillna('') + '. Material: ' +
    df['material'].fillna('') + '. Color: ' +
    df['color'].fillna('')
)
# Generate embeddings for the combined text
# This may take a few minutes depending on your hardware
text_embeddings = text_embedding_model.encode(df['combined_text'].tolist(), show_progress_bar=True)
print(f"Generated text embeddings with shape: {text_embeddings.shape}")
# Expected output shape: (210, 384)

2025-10-18 08:30:12.088687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760776212.112858     120 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760776212.119852     120 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `ty

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Generated text embeddings with shape: (210, 384)


In [5]:
import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import ast
from tqdm import tqdm
import pandas as pd

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained ResNet50 and remove final FC layer
resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
resnet50.eval()
# keep everything except the final fully-connected layer
image_embedding_model = torch.nn.Sequential(*list(resnet50.children())[:-1])
image_embedding_model.to(device)
image_embedding_model.eval()

# Preprocessing (ImageNet)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

def get_image_embedding(image_url):
    """Download, preprocess and return a 1D numpy embedding of length 2048.
       Returns np.zeros(2048) on failure."""
    if not isinstance(image_url, str) or not image_url:
        return np.zeros(2048, dtype=np.float32)
    try:
        response = requests.get(image_url.strip(), timeout=10)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content)).convert('RGB')
        img_t = preprocess(img).unsqueeze(0).to(device)  # shape (1,3,224,224)
        with torch.no_grad():
            emb = image_embedding_model(img_t)           # shape (1,2048,1,1)
            emb = emb.view(emb.size(0), -1)             # shape (1,2048)
            emb = emb.cpu().numpy().squeeze(0)         # shape (2048,)
        return emb.astype(np.float32)
    except Exception as e:
        # you can replace print with logging if you prefer
        print(f"Could not process image {image_url!r}: {e}")
        return np.zeros(2048, dtype=np.float32)

def get_first_image_url(row):
    """Return first URL from row['images'].
       Supports: actual list, string representation of list, single URL string, NaN."""
    val = row.get('images') if isinstance(row, dict) else row['images']
    # handle NaN / None
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return None
    # If it's already a list, return first element
    if isinstance(val, list):
        return val[0].strip() if val else None
    # If it's a string, try to parse or use it directly
    if isinstance(val, str):
        s = val.strip()
        # If string looks like a python list e.g. "['url1', 'url2']"
        if s.startswith('[') and s.endswith(']'):
            try:
                parsed = ast.literal_eval(s)
                if isinstance(parsed, list) and parsed:
                    return parsed[0].strip()
            except (ValueError, SyntaxError):
                pass
        # otherwise assume it's a single URL
        return s
    # else
    return None

# Example: assume df is already a pandas DataFrame with 'images' column
# df['first_image_url'] = df.apply(get_first_image_url, axis=1)  # axis=1 if row-by-row
# If df is large, vectorized approach (faster) — but apply is fine for moderate size.

df['first_image_url'] = df.apply(get_first_image_url, axis=1)

# Generate embeddings (with progress bar). This will create shape (N, 2048)
urls = df['first_image_url'].tolist()
image_embeddings = np.zeros((len(urls), 2048), dtype=np.float32)
for i, url in enumerate(tqdm(urls, desc="Embedding images")):
    image_embeddings[i] = get_image_embedding(url)

print(f"Generated image embeddings with shape: {image_embeddings.shape}")


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 198MB/s] 
Embedding images: 100%|██████████| 210/210 [00:26<00:00,  8.04it/s]

Generated image embeddings with shape: (210, 2048)


In [6]:
# Ensure both arrays have the same number of rows (products)
assert text_embeddings.shape[0] == image_embeddings.shape[0], \
    f"Row mismatch: text={text_embeddings.shape[0]} vs image={image_embeddings.shape[0]}"

# Concatenate along feature dimension
multi_modal_embeddings = np.concatenate([text_embeddings, image_embeddings], axis=1)

print(f"Generated multi-modal embeddings with shape: {multi_modal_embeddings.shape}")
# Expected output: (210, 2432)


Generated multi-modal embeddings with shape: (210, 2432)


In [8]:
# Install the official Pinecone package
!pip uninstall -y pinecone-client
!pip install -U pinecone


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 10.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible

In [10]:
# Install the latest Pinecone SDK (not pinecone-client)
# !pip -q install -U pinecone tqdm

from pinecone import Pinecone, ServerlessSpec
from kaggle_secrets import UserSecretsClient
from tqdm import tqdm
import numpy as np
import math

# 1) Init client (Kaggle secret)
api_key = UserSecretsClient().get_secret("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

# 2) Create a serverless index if needed
index_name = "product-recommendations"
embedding_dim = int(multi_modal_embeddings.shape[1])  # 2432

if index_name not in pc.list_indexes().names():
    print(f"Creating new serverless index '{index_name}'...")
    pc.create_index(
        name=index_name,
        dimension=embedding_dim,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    print("Index created.")
else:
    print(f"Index '{index_name}' already exists.")

# (optional) wait until ready
desc = pc.describe_index(index_name)
index = pc.Index(host=desc.host)

# 3) Prepare data for upserting (dict format)
def _clean_meta(d):
    out = {}
    for k, v in d.items():
        if v is None:
            continue
        if isinstance(v, float) and np.isnan(v):
            continue
        out[k] = v
    return out

vectors_to_upsert = []
# reset_index ensures 'i' aligns with multi_modal_embeddings row i
for i, row in df.reset_index(drop=True).iterrows():
    vector_id = str(row.get("uniq_id", i))
    values = multi_modal_embeddings[i].astype(float).tolist()
    metadata = _clean_meta({
        "title": row.get("title"),
        "brand": row.get("brand"),
        "price": row.get("price"),
        "image_url": row.get("first_image_url"),
    })
    vectors_to_upsert.append({"id": vector_id, "values": values, "metadata": metadata})

# 4) Upsert in batches
batch_size = 100
print("Upserting vectors to Pinecone...")
for start in tqdm(range(0, len(vectors_to_upsert), batch_size)):
    batch = vectors_to_upsert[start:start + batch_size]
    index.upsert(vectors=batch)

# 5) Check stats
print("\nIndex stats:")
print(index.describe_index_stats())


Creating new serverless index 'product-recommendations'...
Index created.
Upserting vectors to Pinecone...


100%|██████████| 3/3 [00:02<00:00,  1.11it/s]



Index stats:
{'dimension': 2432,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


In [11]:
import time

# Wait for 30 seconds to allow the index to update
print("Waiting for 30 seconds for the index to update...")
time.sleep(30)

# Check the stats again
print("\nUpdated Index stats:")
print(index.describe_index_stats())

Waiting for 30 seconds for the index to update...

Updated Index stats:
{'dimension': 2432,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 210}},
 'total_vector_count': 210,
 'vector_type': 'dense'}


In [1]:
# PREREQS (run once)
!pip -q install -U "pinecone==5.*" "transformers==4.46.2" "accelerate>=0.34.2" "sentence-transformers>=3.1.1" "tqdm"

#  CONNECT TO PINECONE
from kaggle_secrets import UserSecretsClient
from pinecone import Pinecone
import pandas as pd

user_secrets = UserSecretsClient()
PINECONE_API_KEY = user_secrets.get_secret("PINECONE_API_KEY")

pc = Pinecone(api_key=PINECONE_API_KEY)
INDEX_NAME = "product-recommendations"

# Use data-plane host for speed
idx_desc = pc.describe_index(INDEX_NAME)
index = pc.Index(host=idx_desc.host)

stats = index.describe_index_stats()
print("Index connected. Stats:", stats)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 6.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 73.3 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.6/486.6 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 64.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s

In [2]:
# RETRIEVAL HELPERS

from sentence_transformers import SentenceTransformer

TEXT_DIM, IMG_DIM = 384, 2048
enc = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def encode_query_mm(q: str, w_text: float = 1.0):
    v = enc.encode(q, normalize_embeddings=True).tolist()
    v = [w_text * x for x in v]
    return v + [0.0] * IMG_DIM


def search(query: str, top_k: int = 5, w_text: float = 1.0, filt: dict | None = None) -> pd.DataFrame:
    qvec = encode_query_mm(query, w_text=w_text)
    res = index.query(vector=qvec, top_k=top_k, include_metadata=True, filter=filt or {})
    rows = []
    for m in res.get("matches", []):
        md = m.get("metadata", {})
        rows.append({
            "id": m.get("id"),
            "score": float(m.get("score", 0.0)),
            "title": md.get("title"),
            "brand": md.get("brand"),
            "price": md.get("price"),
            "image_url": md.get("image_url"),
        })
    return pd.DataFrame(rows)

# Smoke test
hits = search("sofa", top_k=5)
print(hits.head(3))


2025-10-18 09:50:51.939693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760781052.127906      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760781052.180198      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                     id     score  \
0  fe25ae1d-4a82-57ad-9bab-b9de4321fd0b  0.024508   
1  3cbd8443-b3ae-5011-bf59-50f47479a1a7  0.022922   
2  4184968f-0344-5a58-8c95-1bb6462b95b5  0.022527   

                                               title            brand   price  \
0  Karl home Accent Chair Mid-Century Modern Chai...  Karl home Store  149.99   
1  pranovo Metal Sofa Handle Cable Recliner Chair...          pranovo   13.50   
2  DBTHTSK Sofa Latch,Bed Replacement Parts,Heavy...          DBTHTSK   12.99   

                                           image_url  
0  https://m.media-amazon.com/images/I/51+a05Mxh+...  
1  https://m.media-amazon.com/images/I/3144eTNpeE...  
2  https://m.media-amazon.com/images/I/41gQlYHLvc...  


In [5]:
# LOAD GEMMA 2B IT 
import os, torch
from kaggle_secrets import UserSecretsClient
from transformers import AutoTokenizer, AutoModelForCausalLM

os.environ["TOKENIZERS_PARALLELISM"] = "false"

user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")  # ensure you've accepted access on HF

MODEL_ID = "google/gemma-2b-it"
use_gpu = torch.cuda.is_available()
torch_dtype = torch.float16 if use_gpu else torch.float32  # T4 => fp16

tok = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token

gen = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    token=HF_TOKEN,
    device_map="auto",
    torch_dtype=torch_dtype,   # <-- key change
)

print("Gemma 2B IT loaded on", "GPU (fp16)" if use_gpu else "CPU (fp32)")


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Gemma 2B IT loaded on GPU (fp16)


In [38]:
import re, torch

def gemma_answer(query: str, df_hits: pd.DataFrame, max_new_tokens: int = 240) -> str:
    titles = [str(t).strip() for t in df_hits.get("title", []).fillna("").tolist() if str(t).strip()]
    ctx = []
    for _, r in df_hits.head(5).iterrows():
        ctx.append(f"- {r.get('title','N/A')} (Brand: {r.get('brand','N/A')}, Price: {r.get('price','N/A')})")
    ctx = "\n".join(ctx) if ctx else "No context."

    prompt = (
        "You are a concise, helpful product recommendation assistant.\n"
        "Rules (follow strictly):\n"
        "- Never say you cannot answer; if the exact keyword is missing, pick the closest relevant items from the context (chairs/sofas/ottomans/benches/trays) and still answer.\n"
        "- Do NOT start with 'Sure', 'Okay', or 'Here is/Here’s'. No emojis or meta-chat.\n"
        "- Write exactly ONE paragraph of 4–6 sentences. Start neutrally (not a brand).\n"
        "- Mention at least two product titles exactly as in the context. Use only the context.\n\n"
        f"Context:\n{ctx}\n\n"
        f"User need:\n{query}\n\n"
        "Write now."
    )

    ins = tok(prompt, return_tensors="pt").to(gen.device)
    with torch.no_grad():
        out = gen.generate(
            input_ids=ins["input_ids"],
            attention_mask=ins.get("attention_mask"),
            max_new_tokens=max_new_tokens,
            min_new_tokens=min(120, max_new_tokens-20),
            do_sample=False,
            temperature=0.0,
            repetition_penalty=1.05,
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.eos_token_id,
        )

    new_tokens = out[0, ins["input_ids"].shape[1]:]
    txt = tok.decode(new_tokens, skip_special_tokens=True).strip()

    # sanitize openers + meta
    txt = re.sub(r"^(sure,?\s*|okay,?\s*|here'?s\s+.*?:\s*)", "", txt, flags=re.I).strip()
    for b in [r"i cannot answer", r"i can't", r"unable", r"not mention", r"no context",
              r"i'm here to assist", r"would you like", r"let me know", r"please note",
              r"i hope this helps", r"[😊😁🙂😉👍]"]:
        txt = re.sub(b, "", txt, flags=re.I)
    txt = re.sub(r"\s+", " ", txt).strip()

    # ensure ≥2 titles mentioned
    needed = []
    for t in titles[:3]:
        if t and t not in txt:
            needed.append(t)
        if len(needed) >= 2:
            break
    if needed:
        txt += " In particular, consider " + " and ".join(f'\"{n}\"' for n in needed[:2]) + "."

    # clamp to 4–6 sentences
    sents = [s.strip() for s in re.split(r"(?<=[.!?])\s+", txt) if s.strip()]
    while len(sents) < 4:
        sents.append("These options balance comfort, value, and everyday usability at home.")
    txt = " ".join(sents[:6])

    # refusal guard: if still refused or <2 titles, build deterministic fallback
    if re.search(r"(cannot|can't|unable|no context|not mention)", txt, re.I) or sum(1 for t in titles if t in txt) < 2:
        picks = titles[:3]
        if len(picks) >= 2:
            base = (
                "These options offer practical seating and storage for living spaces. "
                f"\"{picks[0]}\" and \"{picks[1]}\" stand out for their everyday comfort and value"
                + (f", while \"{picks[2]}\" adds a versatile accent." if len(picks) > 2 else ".")
            )
        elif len(picks) == 1:
            base = f"\"{picks[0]}\" is a practical choice for compact living spaces with solid everyday value."
        else:
            base = "These options balance comfort, value, and everyday usability at home."
        txt = base

    return txt


In [39]:
# filter_hits 
import pandas as pd, re

BAD = ("lever","latch","cable","release","hardware","bracket","replacement","webbing","band","repair","modification")
GOOD = ("sofa","chair","ottoman","bench","couch","table","tray","armchair","stool")

def filter_hits(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: 
        return df
    t = df["title"].fillna("").str.lower()
    keep = t.apply(lambda x: any(g in x for g in GOOD) and not any(b in x for b in BAD))
    df2 = df[keep].copy()
    if len(df2) >= 2:
        return df2.head(5)
    util = t.str.contains(r"(tray|table|ottoman|stool)", regex=True) & ~t.apply(lambda x: any(b in x for b in BAD))
    extra = df[util].copy()
    out = pd.concat([df2, extra]).drop_duplicates(subset=["id"])
    return out.head(5) if len(out) else df.head(5)


In [40]:
#RAG
import json

def _expand_query(q: str) -> str:
    ql = q.lower()
    if "sofa" in ql:
        return "sofa couch chair ottoman bench living room seating"
    return q

def rag(query: str, top_k: int = 8) -> dict:
    raw = search(_expand_query(query), top_k=top_k)
    used = filter_hits(raw) if 'filter_hits' in globals() else raw
    text = gemma_answer(query, used)
    return {"recommendations": used.to_dict(orient="records"), "generated_text": text}

sample = rag("sofa", top_k=8)
print(sample["generated_text"][:500])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


These options offer practical seating and storage for living spaces. "Karl home Accent Chair Mid-Century Modern Chair with Pillow Upholstered Lounge Arm Chair with Solid Wood Frame & Soft Cushion for Living Room, Bedroom, Belcony, Beige" and "Nalupatio Storage Ottoman, Bedroom End Bench，Upholstered Fabric Storage Ottoman with Safety Hinge, Entryway Padded Footstool, Ottoman Bench for Living Room & Bedroom(Light Green)" stand out for their everyday comfort and value, while "Phantoscope Storage Ot


In [41]:

out_path = "/kaggle/working/sample_rag_response.json"
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(sample, f, ensure_ascii=False, indent=2)
print("Saved:", out_path)


Saved: /kaggle/working/sample_rag_response.json
